In [ ]:
from ngsolve.webgui import Draw

import numpy as np
import ngsolve as ng
import dill
import pickle

motor_npz = np.load('../meshes/motor_pizza.npz', allow_pickle = True)

geoOCC = motor_npz['geoOCC'].tolist()
m = motor_npz['m']; m_new = m
j3 = motor_npz['j3']

geoOCCmesh = geoOCC.GenerateMesh()
mesh = ng.Mesh(geoOCCmesh)
# mesh.Refine()

# Draw(mesh)

In [ ]:
# import sys
# sys.path.insert(0,'../../') # adds parent directory

# import pde
# order = 2

# MESH = pde.mesh.netgen(mesh.ngmesh)

# phi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'phi', order = x)
# curlphi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'curlphi', order = x)

# D = lambda x : pde.int.assemble(MESH, order = x)

# myM = lambda x: phi_Hcurl(x)[0] @ D(x) @ phi_Hcurl(x)[0].T + \
#                 phi_Hcurl(x)[1] @ D(x) @ phi_Hcurl(x)[1].T

# myK = lambda x: curlphi_Hcurl(x) @ D(x) @ curlphi_Hcurl(x).T

# R = pde.tools.tree_cotree_gauge(MESH)

In [64]:
HCurl = ng.HCurl(mesh, order = 1, nograds = True, dirichlet = 'stator_outer')

u,v = HCurl.TnT()

K = ng.BilinearForm(HCurl)
K += (ng.curl(u)*ng.curl(v)+0*u*v)*ng.dx
K.Assemble()

f = ng.LinearForm(HCurl)
f += ng.CF(1)*ng.curl(v)*ng.dx
# f += ng.CF((1,1))*v*ng.dx
f.Assemble()

# rows,cols,vals = K.mat.COO()
# import scipy.sparse as sp
# ngK = sp.csr_matrix((vals,(rows,cols)))

# np.linalg.matrix_rank(ngK.A)
# print(ngK.shape)

In [65]:
# List all edges... this is still kinda slow but what can you do
i = 0
edges = []
for edge in mesh.edges:
    index = HCurl.FreeDofs()[i]
    edge = [int(str(edge.vertices[0])[1:]), int(str(edge.vertices[1])[1:])]
    i = i + 1
    if index == 1:
        # Append only FreeDof edges. This eliminates Dirichlet edges.
        edges.append(edge)

In [66]:
from mst4 import *
g = Graph(MESH.np)
g.addEdges(edges)
mst = g.KruskalMST()

tree_indices = np.array(mst)
cotree_indices = np.setdiff1d(np.arange(MESH.NoEdges),tree_indices)
# print(MESH.NoEdges,len(mst),MESH.np,tree_indices.size,cotree_indices.size)

In [71]:
import pyngcore as pyng
FreeDofsCotreeGauge = pyng.BitArray(HCurl.FreeDofs())

for _,j in enumerate(cotree_indices): FreeDofsCotreeGauge.Clear(j)
# print(FreeDofsCotreeGauge)

In [72]:
x = ng.GridFunction(HCurl)
x.vec.data = K.mat.Inverse(freedofs = FreeDofsCotreeGauge, inverse = 'sparsecholesky')*f.vec
# x.vec.data = K.mat.Inverse(HCurl.FreeDofs(), inverse = 'sparsecholesky')*f.vec

Draw(x)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:


edges = MESH.EdgesToVertices[:,:2]

# for i in range(edges.shape[0]): g.addEdge(edges[i,0],edges[i,1],i)



mst = g.KruskalMST()


# tree_indices = np.array(g.MST)[:,2]
# cotree_indices = np.setdiff1d(np.arange(MESH.NoEdges),tree_indices)

In [ ]:
# for edge in mesh.edges:
#     print(edge.vertices)

print(MESH.EdgesToVertices)

# Generate or load a mesh
# mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))


# # Print edge list
# for i, e in enumerate(edges):
#     print(f"Edge {i}: connects vertex {e[0]} to vertex {e[1]}")
edges